# Coastal Compound Floods

## TODO
- Use monthly median and daily mean
- Check extremes

## Background

**Converted all to daily to match cmip6 $\rightarrow$ daily max/mean from hourly and original cmip6**

#### Observational data
- Max/mean wind Data (m/s) from noaa (6 min)
- Max/mean tides and Water Level (m) from noaa (6 min)
- Max/Mean wind speed, mean direction, total precip from meteostat (1 hr)

#### CMIP6 only using ssp 245
- sfcwind: LARC_sfcWind_historical_daily.csv, LARC_sfcWind_ssp245_daily.csv
- pr: LARC_pr_historical_daily.csv, LARC_pr_ssp126_daily.csv, LARC_pr_ssp245_daily.csv, LARC_pr_ssp370_daily.csv
    - The sum of liquid and frozen water, comprising rain and snow, that falls to the Earth's surface. It is the sum of large-scale precipitation and convective precipitation. This parameter does not include fog, dew or the precipitation that evaporates in the atmosphere before it lands at the surface of the Earth. This variable represents amount of water per unit area and time.



### Modifications and Assumptions
**General**
- Rows with less than n observations per year were removed
- Only 1950 and after
- Set all 0 precip to nan

**Wind**
- wind direction degree [0, 360]
- wind gust >= speed >= 0
- Extremes: max speed and gust in 1 hour (based on 6 minute intervals)
- Circular mean wind direction (max mode for text version)

**Tides**
- MHHW and MSL are most indicative of flood risks
- high_low tides (HH, H, L, LL) are all the same across MHHW, MHW, MSL… → only processed MSL to get labeled dates
- Value - Measured water level height for product='high_low' and product='water_level' for the same datum are different → use values in water_level
- Sigma - Standard deviation of 1 second samples used to compute the water level height; **sigma > 3** is considered bad quality and removed


<details>
<summary> NOAA Tides & Currents: mean in past time interval </summary>
    
- Virginia: https://tidesandcurrents.noaa.gov/map/index.html?region=Virginia#
- Yorktown: https://tidesandcurrents.noaa.gov/inventory.html?id=8637689
- Sewells Point: https://tidesandcurrents.noaa.gov/inventory.html?id=8638610
- Gloucester before 2003: https://tidesandcurrents.noaa.gov/inventory.html?id=8637624
- https://pypi.org/project/noaa-coops/
- product: https://api.tidesandcurrents.noaa.gov/api/prod/#products
- datum: https://api.tidesandcurrents.noaa.gov/api/prod/#datum:~:text=for%20the%20station-,Datum,-The%20datum%20can
- Output columns: https://api.tidesandcurrents.noaa.gov/api/prod/responseHelp.html
- https://maps.waterdata.usgs.gov/mapper/index.html
- dates of active stations are from (first available day) to (at most day of download)
</details>

<details>
<summary> Meteostat: measurements at end of each hour </summary>
    
- https://dev.meteostat.net/python/daily.html#data-structure
- Variables and units: https://dev.meteostat.net/formats.html
- Station data columns: https://dev.meteostat.net/python/stations.html#data-structure
</details>

<details>
<summary> 2016 Floods </summary>
    
- September 2-3, 2016 Tropical Storm Hermine https://www.weather.gov/mhx/hermine_090216
- September 11 – 23, 2016: Tropical Storm Julia https://www.wpc.ncep.noaa.gov/tropical/rain/julia2016.html
- October 8, 2016: Hurricane Matthew https://www.weather.gov/ilm/matthew
</details>

In [1]:
# !pip install noaa_coops
# !pip install meteostat

import os
import re
import glob
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.colors import qualitative
from plotly.subplots import make_subplots
import geopandas as gpd
from shapely.geometry import Point, box
import contextily as cx
import matplotlib.dates as mdates
import matplotlib.lines as mlines
from IPython.display import Image, display
import plot_settings
from statsmodels.tsa.seasonal import STL
from scipy.signal import periodogram


warnings.filterwarnings('ignore')

static = True # interactive/static plotly plots
PATH = 'data/floods'
PATH2 = PATH + '/hourly' # daily_max, daily_mean, hourly
# !python process_flood.py --output_dir $PATH

# Load Data

In [2]:
# Load NOAA and Meteo
noaa = pd.read_csv(f'{PATH}/noaa_tc.csv')
noaa['station_id'] = noaa.station_id.astype(str)
noaa[['start_date', 'end_date']] = noaa[['start_date', 'end_date']].apply(pd.to_datetime)
noaa['station'] = noaa.station_name + ' ' + noaa.station_id
wind_stations = noaa.loc[noaa.variable == 'Wind', 'station_id'].values
water_stations = noaa.loc[noaa.variable == 'Verified 6-Minute Water Level', 'station_id'].values

meteo = pd.read_csv(f'{PATH}/meteo.csv').rename(columns={'latitude': 'lat', 'longitude': 'lon'})
meteo[['hourly_start', 'hourly_end']] = meteo[['hourly_start', 'hourly_end']].apply(pd.to_datetime)
meteo['station'] = meteo.name + ' ' + meteo.id

# Load DataFrames
wind_df = pd.read_parquet(f'{PATH2}/noaa_tc.wind.parquet')
wind_df = wind_df.loc[:, ~wind_df.columns.str.contains('flag')]
water_df = pd.read_parquet(f'{PATH2}/noaa_tc.water.parquet')
meteo_df = pd.read_parquet(f'{PATH2}/meteo.pr_wind.parquet')

df = pd.concat([wind_df, water_df, meteo_df], axis=1)
df = df[sorted(df.columns)][df.index.year >= 1950]
# df[df.filter(regex='pr_').columns] = df.filter(regex='pr_').replace(0, np.nan)

# Remove sparse years
def mask_sparse_years(df, min_obs=100):
    '''Set all values in a year to nan if that year does not have at least min_obs values'''
    sparse_mask = df.notna().groupby(df.index.year).sum() < min_obs
    year_index = df.index.year.values
    mask = np.zeros_like(df.values, dtype=bool)
    for col_idx, col in enumerate(df.columns):
        for year_idx, year in enumerate(sparse_mask.index):
            if sparse_mask.iloc[year_idx, col_idx]:
                mask[year_index == year, col_idx] = True
    return df.mask(mask)

df = mask_sparse_years(df).reindex(pd.date_range(df.index.min(), df.index.max(), freq='h'))

# Variables map
var_map = {
    'speed_': 'Wind Speed (m/s)',
    'gust_': 'Wind Gust (m/s)',
    'mhhw_|msl_': 'Mean Higher High Water and Mean Sea Level (m)',
    'mhhw_': 'Mean Higher High Water (m)',
    'msl_': 'Mean Sea Level (m)',
    'pr_': 'Precipitation (mm)',
    'dir_': 'Wind Direction',
    'dirdeg_': 'Wind Direction (degree)',
    'type_': 'Tide Type (LL-1, L-2, H-3, HH-4)'
}

# Stations with color
color_list = px.colors.qualitative.Plotly + px.colors.qualitative.Light24 + px.colors.qualitative.Alphabet
colors = {}
for sid, name in sorted(
    list(zip(noaa['station_id'].astype(str), noaa['station_name'])) +
    list(zip(meteo['id'], meteo['name']))
):
    colors.setdefault(sid, {'name': name or sid, 'color': color_list[len(colors)]})
colors = pd.DataFrame.from_dict(colors, orient='index').reset_index().rename(columns={'index': 'id'})
tide_map = {1: 'LL', 2: 'L', 3: 'H', 4: 'HH'}
ordered_tides = list(tide_map.values())[::-1]

In [3]:
import pandas as pd
import numpy as np
from scipy import stats

def standardize_climatology(df, variables, ref_station=None):
    result = df.copy()
    for var in variables:
        cols = [c for c in df if var in c]
        if not cols: continue
        ref_col = f"{var}{ref_station}" if ref_station and f"{var}{ref_station}" in cols else max(cols, key=lambda c: df[c].count())
        ref = df[ref_col].dropna()
        if ref.size < 100: continue
        for c in cols:
            if c == ref_col: continue
            src = df[c].dropna()
            if src.size < 100: continue
            common = src.index.intersection(ref.index)
            if common.size < 100:
                ranks = stats.rankdata(src) / src.size
                result.loc[src.index, c] = np.quantile(ref, ranks)
            else:
                src_vals, ref_vals = src.loc[common], ref.loc[common]
                if any(k in c for k in ['pr_', 'speed_', 'gust_']):
                    idx = (src_vals > 0) & (ref_vals > 0)
                    if idx.sum() > 50:
                        result.loc[src.index, c] = src * (ref_vals[idx].median() / src_vals[idx].median())
                else:
                    result.loc[src.index, c] = src + (ref_vals.median() - src_vals.median())
    return result

def analyze_precipitation_with_storms(df, pr_pattern='pr_', min_gap=3):
    pr_df = df.filter(regex=pr_pattern).dropna(how='all')
    labeled_df, all_storms = df.copy(), {}
    for col in pr_df:
        storms, mask = [], pd.Series(np.nan, index=pr_df.index)
        in_storm, start, end, total, dry = False, None, None, 0, 0
        for t, v in pr_df[col].items():
            if pd.notna(v) and v > 0:
                if not in_storm: in_storm, start = True, t
                end, total, dry = t, total + v, 0
            elif in_storm:
                dry += 1
                if dry >= min_gap:
                    hrs = (end - start).total_seconds() / 3600 + 1
                    intensity = total/hrs
                    storms.append({'start': start, 'end': end, 'hours': hrs, 'mm': total, 'intensity': intensity})
                    mask[start:end] = intensity
                    in_storm, total = False, 0
        if in_storm:
            hrs = (end - start).total_seconds() / 3600 + 1
            intensity = total/hrs
            storms.append({'start': start, 'end': end, 'hours': hrs, 'mm': total, 'intensity': intensity})
            mask[start:end] = intensity
        labeled_df[f'intensity_{col}'] = mask
        all_storms[col] = pd.DataFrame(storms)
    stats = {
        col.replace(pr_pattern, ''): {
            'storms': len(s),
            'total_mm': s['mm'].sum() if not s.empty else 0,
            'avg_intensity': s['intensity'].mean() if not s.empty else 0,
            'max_mm': s['mm'].max() if not s.empty else 0,
            'max_intensity': s['intensity'].max() if not s.empty else 0
        } for col, s in all_storms.items()
    }
    return pd.DataFrame(stats).T, all_storms, labeled_df


def identify_compound_events(df, storm_data, storms_df, daily_precip, thresholds, window_hours=24):
    """    
    Parameters:
    - df: DataFrame with climate data
    - storm_data: DataFrame with calculated storm intensity values
    - storms_df: Dictionary with storm events by station
    - daily_precip: DataFrame with daily precipitation totals
    - thresholds: Dictionary with threshold values
    - window_hours: Hours to consider for coincident events
    """
    events = pd.DataFrame(index=df.index)
    
    # High precipitation intensity events
    events['high_intensity'] = storm_data.filter(regex='intensity_').max(axis=1) >= thresholds.get('intensity', 8.0)
    
    # High daily precipitation totals
    max_daily = daily_precip.max(axis=1)
    levels = [100, 150, 200]
    for i, t in enumerate(levels, 1):
        events[f'daily_precip_{t}mm'] = max_daily >= t
    events['high_daily_precip'] = sum((max_daily >= t).astype(int) for t in levels)
    events['high_daily_precip_flag'] = events['high_daily_precip'] > 0

    # High tide events
    events['high_tide'] = df.filter(regex='mhhw_').max(axis=1) > thresholds.get('mhhw', 0.3)
    
    # Storm surge potential from wind direction and speed    
    dir_df = df.filter(regex='^dirdeg_')
    onshore = ((dir_df >= 0) & (dir_df <= 135)) | ((dir_df >= 300) & (dir_df <= 360))
    onshore = onshore.any(axis=1)
    wind = pd.Series(False, index=df.index)
    for p, k, d in [('speed_', 'wind_speed', 10), ('gust_', 'wind_gust', 15)]:
        wind_df = df.filter(regex=f'^{p}')
        if not wind_df.empty:
            wind |= wind_df.max(axis=1) > thresholds.get(k, d)
    events['storm_surge'] = onshore & wind

    # Calculate windows for each hazard
    events['compound_risk'] = 0
    risk_cols = []
    
    # Define the main hazards we're looking for in compound events
    main_hazards = ['high_intensity', 'high_daily_precip_flag', 'high_tide', 'storm_surge']
    active_hazards = [h for h in main_hazards if h in events.columns]
    
    for hazard in active_hazards:
        window_col = f'{hazard}_window'
        window = pd.Series(0, index=df.index)
        hazard_events = events[hazard].fillna(False)
        
        for idx in events.index[hazard_events]:
            end_idx = min(events.index[-1], idx + pd.Timedelta(hours=window_hours))
            window.loc[idx:end_idx] = 1
            
        events[window_col] = window
        risk_cols.append(window_col)
    
    # Calculate compound risk as sum of overlapping hazard windows
    if risk_cols:
        events['compound_risk'] = events[risk_cols].sum(axis=1)
    
    # Extract compound flood events (at least 2 hazards coincident)
    compound_events = []
    current_event = None
    
    for idx, row in events.iterrows():
        if row['compound_risk'] >= 2:
            if current_event is None:
                current_event = {'start': idx, 'hazards': set()}
                
            current_event['end'] = idx
            
            for hazard in active_hazards:
                window_col = f'{hazard}_window'
                if window_col in row and row[window_col]:
                    current_event['hazards'].add(hazard)
        
        elif current_event is not None and 'end' in current_event:
            duration = (current_event['end'] - current_event['start']).total_seconds() / 3600
            
            # Extract the maximum daily precipitation value during this event
            if 'high_daily_precip' in events.columns:
                max_daily_level = events.loc[current_event['start']:current_event['end'], 'high_daily_precip'].max()
            else:
                max_daily_level = 0
                
            compound_events.append({
                'start': current_event['start'],
                'end': current_event['end'],
                'duration': duration,
                'hazard_count': len(current_event['hazards']),
                'hazards': list(current_event['hazards']),
                'daily_precip_level': max_daily_level
            })
            
            current_event = None
    
    if current_event is not None and 'end' in current_event:
        duration = (current_event['end'] - current_event['start']).total_seconds() / 3600
        
        if 'high_daily_precip' in events.columns:
            max_daily_level = events.loc[current_event['start']:current_event['end'], 'high_daily_precip'].max()
        else:
            max_daily_level = 0
            
        compound_events.append({
            'start': current_event['start'],
            'end': current_event['end'],
            'duration': duration,
            'hazard_count': len(current_event['hazards']),
            'hazards': list(current_event['hazards']),
            'daily_precip_level': max_daily_level
        })
    
    return pd.DataFrame(compound_events) if compound_events else pd.DataFrame(), events

def assess_flood_risk(df, storms, events):
    """
    Assess coastal flood risk using precipitation intensity, daily totals and compound events.
    """
    if events.empty: 
        return pd.DataFrame()
    
    risk = events.copy()
    if 'daily_precip_level' not in risk.columns:
        risk['daily_precip_level'] = 0
        
    risk['mean_precip_mm'] = 0  # Mean precipitation during the event
    risk['mean_precip_intensity'] = 0  # Mean intensity during the event
    
    for i, event in risk.iterrows():
        start_time, end_time = event['start'], event['end']
        
        # Find overlapping storms
        event_precip_values = []
        intensity_values = []
        
        for station, storm_df in storms.items():
            overlapping = storm_df[(storm_df['start'] <= end_time) & (storm_df['end'] >= start_time)]
            if not overlapping.empty:
                # Collect precipitation and intensity values for mean calculation
                event_precip_values.extend(overlapping['mm'].tolist())
                intensity_values.extend(overlapping['intensity'].tolist())
        
        if event_precip_values:
            risk.loc[i, 'mean_precip_mm'] = np.mean(event_precip_values)
        if intensity_values:
            risk.loc[i, 'mean_precip_intensity'] = np.mean(intensity_values)
    
    # Calculate risk score based on hazard count, intensity and daily precipitation
    risk['intensity_score'] = 0
    risk.loc[risk['mean_precip_intensity'] >= 8, 'intensity_score'] = 1
    risk.loc[risk['mean_precip_intensity'] >= 12, 'intensity_score'] = 2
    risk.loc[risk['mean_precip_intensity'] >= 16, 'intensity_score'] = 3
    risk['risk_score'] = (risk['hazard_count'] + risk['daily_precip_level'] + risk['intensity_score'])
    risk['risk_level'] = pd.cut(risk['risk_score'], bins=[0, 3, 5, 7, 100], labels=['Low', 'Moderate', 'High', 'Extreme'])
    return risk


def analyze_compound_coastal_floods(df):
    variables = ['pr_', 'speed_', 'gust_', 'dirdeg_', 'mhhw_', 'msl_']
    std_df = standardize_climatology(df, variables)
    stats, storms, labeled = analyze_precipitation_with_storms(std_df)
    daily_pr = std_df.filter(regex='pr_').rolling('24H').sum()

    thresholds = {
        'intensity': 8.0,         # mm/hour (rainfall intensity)
        'mhhw': 0.3,              # m above mean higher high water
        'wind_speed': 10,         # m/s 
        'wind_gust': 15           # m/s
    }
    
    events, indicators = identify_compound_events(std_df, labeled, storms, daily_pr, thresholds)
    risk = assess_flood_risk(std_df, storms, events)
    
    return risk, events, indicators, std_df, stats

risk_assessment, compound_events, event_indicators, std_df, stats = analyze_compound_coastal_floods(df)

print(f"Identified {len(compound_events)} compound coastal flood events")
print("\nPrecipitation Statistics:")
print(stats)

if not risk_assessment.empty:
    print("\nRisk Assessment Summary:")
    print(f"Events by risk level:")
    print(risk_assessment['risk_level'].value_counts())
    
    print("\nTop 5 highest risk events:")
    print(risk_assessment.sort_values(by='risk_score', ascending=False).head(5)[
        ['start', 'end', 'duration', 'hazard_count', 'mean_precip_mm', 
         'mean_precip_intensity', 'daily_precip_level', 'risk_score', 'risk_level']
    ])
else:
    print("\nNo compound flood events identified with the current thresholds.")

Identified 510 compound coastal flood events

Precipitation Statistics:
       storms      total_mm  avg_intensity      max_mm  max_intensity
69536   479.0   3123.314286       0.857373  125.828571      10.457143
72308  6575.0  57691.700000       1.706043  231.900000      42.700000
74598  2568.0  20952.000000       1.500770  269.600000     122.300000
KFAF0  2634.0  24896.600000       1.866654  606.500000     267.700000
KPHF0  3399.0  25525.200000       1.447672  252.200000      35.300000

Risk Assessment Summary:
Events by risk level:
risk_level
Low         471
Moderate     23
High          8
Extreme       8
Name: count, dtype: int64

Top 5 highest risk events:
                  start                 end  duration  hazard_count  \
135 2008-05-12 02:00:00 2008-05-14 17:00:00      63.0             4   
341 2018-12-05 18:00:00 2018-12-08 09:00:00      63.0             3   
252 2014-09-24 08:00:00 2014-09-28 00:00:00      88.0             4   
195 2011-08-27 05:00:00 2011-08-29 09:00:00    

In [4]:
risk_assessment[risk_assessment.start.dt.year==2016]

,start,end,duration,hazard_count,hazards,daily_precip_level,mean_precip_mm,mean_precip_intensity,intensity_score,risk_score,risk_level
270,2016-01-10 23:00:00,2016-01-11 10:00:00,11.0,2,"[high_tide, storm_surge]",0,0.0000,0.000000,0,2,Low
271,2016-01-23 03:00:00,2016-01-25 06:00:00,51.0,2,"[high_tide, storm_surge]",0,54.4000,1.790343,0,2,Low
272,2016-02-10 20:00:00,2016-02-11 11:00:00,15.0,2,"[high_tide, storm_surge]",0,0.0000,0.000000,0,2,Low
273,2016-03-20 07:00:00,2016-03-22 07:00:00,48.0,2,"[high_tide, storm_surge]",0,2.7300,0.667030,0,2,Low
274,2016-04-07 21:00:00,2016-04-08 21:00:00,24.0,2,"[high_tide, storm_surge]",0,0.3000,0.300000,0,2,Low
275,2016-06-05 07:00:00,2016-06-06 21:00:00,38.0,2,"[high_tide, high_intensity]",0,15.2125,3.573333,0,2,Low
276,2016-06-23 16:00:00,2016-06-24 14:00:00,22.0,2,"[high_intensity, storm_surge]",0,3.6000,0.900000,0,2,Low
277,2016-07-31 17:00:00,2016-07-31 20:00:00,3.0,2,"[high_intensity, storm_surge]",0,4.6000,1.150000,0,2,Low
278,2016-07-31 22:00:00,2016-08-02 01:00:00,27.0,3,"[high_daily_precip_flag, high_intensity, storm...",1,40.0250,10.343750,1,5,Moderate
279,2016-08-02 17:00:00,2016-08-03 21:00:00,28.0,3,"[high_tide, high_daily_precip_flag, high_inten...",0,27.9200,6.479000,0,3,Low


In [26]:
risk_assessment[risk_assessment.start.dt.year==2016]

,start,end,duration,hazard_count,hazards,daily_precip_level,mean_precip_mm,mean_precip_intensity,intensity_score,risk_score,risk_level
270,2016-01-10 23:00:00,2016-01-11 10:00:00,11.0,2,"[high_tide, storm_surge]",0,0.0000,0.000000,0,2,Low
271,2016-01-23 03:00:00,2016-01-25 06:00:00,51.0,2,"[high_tide, storm_surge]",0,54.4000,1.790343,0,2,Low
272,2016-02-10 20:00:00,2016-02-11 11:00:00,15.0,2,"[high_tide, storm_surge]",0,0.0000,0.000000,0,2,Low
273,2016-03-20 07:00:00,2016-03-22 07:00:00,48.0,2,"[high_tide, storm_surge]",0,2.7300,0.667030,0,2,Low
274,2016-04-07 21:00:00,2016-04-08 21:00:00,24.0,2,"[high_tide, storm_surge]",0,0.3000,0.300000,0,2,Low
275,2016-06-05 07:00:00,2016-06-06 21:00:00,38.0,2,"[high_intensity, high_tide]",0,15.2125,3.573333,0,2,Low
276,2016-06-23 16:00:00,2016-06-24 14:00:00,22.0,2,"[high_intensity, storm_surge]",0,3.6000,0.900000,0,2,Low
277,2016-07-31 17:00:00,2016-07-31 20:00:00,3.0,2,"[high_intensity, storm_surge]",0,4.6000,1.150000,0,2,Low
278,2016-07-31 22:00:00,2016-08-02 01:00:00,27.0,3,"[high_intensity, storm_surge, high_daily_preci...",1,40.0250,10.343750,1,5,Moderate
279,2016-08-02 17:00:00,2016-08-03 21:00:00,28.0,3,"[high_intensity, high_tide, high_daily_precip_...",0,27.9200,6.479000,0,3,Low


In [29]:
risk_assessment.daily_precip_level.value_counts()

daily_precip_level
0    473
1     19
3     11
2      7
Name: count, dtype: int64

In [4]:
storms['pr_72308'].sort_values('intensity')

,start,end,hours,mm,intensity
6391,2023-11-10 18:00:00,2023-11-10 21:00:00,4.0,0.2,0.050000
6509,2024-09-15 18:00:00,2024-09-15 20:00:00,3.0,0.2,0.066667
6536,2024-12-20 10:00:00,2024-12-20 16:00:00,7.0,0.6,0.085714
3893,2004-03-30 19:00:00,2004-03-31 18:00:00,24.0,2.1,0.087500
3302,1998-12-29 06:00:00,1998-12-29 18:00:00,13.0,1.2,0.092308
...,...,...,...,...,...
4199,2006-07-14 21:00:00,2006-07-14 21:00:00,1.0,27.9,27.900000
5454,2016-07-31 22:00:00,2016-08-01 01:00:00,4.0,115.1,28.775000
2760,1994-07-25 22:00:00,1994-07-26 00:00:00,3.0,87.8,29.266667
3480,2000-07-15 13:00:00,2000-07-15 13:00:00,1.0,30.7,30.700000


In [6]:
storms.keys()

dict_keys(['pr_69536', 'pr_72308', 'pr_74598', 'pr_KFAF0', 'pr_KPHF0'])

In [23]:
labeled['pr_69536'].loc['2016-10-08 15:00:00':'2016-10-11 03:00:00'].sum()

np.float64(0.0)

In [27]:
(153+149+231.1+113.7)/4

161.70000000000002